Project plan

1. problem formulation :  
Context : we have been instructed to predict books ratings via a machine learning application from provided data.  
2. Data preprocessing
3. EDA
4. features engineering 
5. modelling & Machine learning 
6. Model validation
7. deployment

In [ ]:
#!conda env list

In [ ]:
# activate environment 
#!conda init bash
#!conda activate Project1

In [ ]:
#pip install statsmodels

In [ ]:
# load libraries 
import pandas as pd
import numpy as np
#import seaborn as sns
#import matplotlib as plt
#import plotly as ply
import sklearn as skl
import datetime as dt
import statsmodels.api as sm
import statsmodels.formula.api as smf
import patsy
import re 
from IPython.display import clear_output, display, HTML, Javascript

In [ ]:
# load data 
df = pd.read_csv("books.csv")

2 Data preprocessing 

2.1  inspect data 

In [ ]:
df.head(2)

In [ ]:
df.tail(2)

In [ ]:
# df shape 
df.shape

In [ ]:
# columns names
df.columns

In [ ]:
# describe numeric variables
df.describe(exclude="object").T

In [ ]:
# describe non-numeric  variables
df.describe(include="object").T

There is no duplicate ID in data as isbn is 11127 the datat set number of cases. 

2.2  clean data 

2.2.1 Clean publication date 

In [ ]:
# change from string to datetime object 
# for loop to parse date 
df["publication_Date"]= pd.Series(dtype="int")
cnt = 0
for i in range(len(df.publication_date)) :
    month,day,year = map(int,df.publication_date[i].split('/'))

    if month in [4,6,9,11] and day > 30:
        day = 30
        df["publication_Date"][i] = dt.date(year,month,day)
    elif(month==2 and day > 28 and year % 4 != 0):
        day = 28
    df["publication_Date"][i] = dt.date(year,month,day)
    cnt +=1
if (cnt == len(df.publication_date)) :
    print("no date issue" )
 

In [ ]:
# remove heading whitespace from num_pages feature 
df = df.rename(columns = {"  num_pages" : "num_pages"})
df.columns

In [ ]:
#na check in the entire dataset 
df.isna().sum().sum()

There is no NA in df

In [ ]:
# na check in average_rating
if (df.average_rating.isna().mean() == 0 ) :
    print("There is no NA in average_rating")
else:
    print("There are NA in average_rating")

3 EDA 

1D EDA 

3.1 Numerical summary and 1D plot

In [ ]:
# numerical summary : average_rating 
df.average_rating.describe()

In [ ]:
# histogram of average ratings 
df.average_rating.hist(bins=60,legend={""})

In [ ]:
# histogram of average ratings :range [0,3.9]
df.average_rating.hist(bins=60,range = [0,3],legend={""})

In [ ]:
# histogram of average ratings :range [0,4.75]
df.average_rating.hist(bins=30,range = [3,4.75],legend={""})

In [ ]:
# histogram of average ratings :range [0,3.9]
df.average_rating.hist(bins=30,range = [4.75,5],legend={""})

In [ ]:
# number of cases where averating are <= 3
df.loc[df.average_rating <= 3].shape

In [ ]:
# number of cases where average rating >= 4.75 
df.loc[df.average_rating > 4.75].shape

In [ ]:
# average ratings sumary statistics 
df.average_rating.describe()

In [ ]:
#average_rating 1st and 3rd quantile 
IQR_average_rating = df.quantile([0.25,0.75])[["average_rating"]]
IQR_average_rating

average_ratings is almost an unimodal distribution with some outliers.
it has median 3.96 and IQR : 0.3 

In [ ]:
#numerical summary num_pages
df.num_pages.describe()

In [ ]:
# histogram of num_pages 
df.num_pages.hist(bins = 60,legend={""})

Num_pages is almost an unimodal distribution, right skewed showing that  at least 50% of books presents 299 pages.

In [ ]:
# numerical summary ratings_count 
(df.ratings_count
.quantile([0.25,0.5,0.75, 1])
)

Maximum rating count is so high that it seems suspect !

In [ ]:
# which book has this ratings count ?
df.title[df.ratings_count == max(df.ratings_count) ]

Twilight is the book with the highest ratings_count.  

In [ ]:


# histogram of ratings_count 
df.ratings_count.hist(bins = 50,range = [0,2000],legend ={""})

Ratings count distribution is unimodal, right skewed. At least 50% of books ratings count is greater or equal to 745.

In [ ]:
# numerical summary publication Date
df.publication_Date.describe()

In [ ]:
(df
["publication_Date"]
.quantile([0.0,0.25,0.5,0.75,1])
)

In [ ]:
# histogram of publication date 
df.publication_Date.hist(bins = 70)

publication date distribution is unimodal and left skewed. At least 50% of books has been published before between 1998 and 2005.

How many books are written in english ?

In [ ]:
# language code 
df3 = df.groupby("language_code").count()
# round 
round(df3.bookID.filter(regex ="^[Ee][Nn][g\\-]")/df.shape[0],2)

95% of book are written in English. The english type being diffrent.  
It may be reasonble to turn language code into a binary variable.  

Who are the five most prolific authors ?

In [ ]:
# authors
(df.groupby("authors")
 .count()
 .sort_values("title",ascending=False)
 [["title"]]
 .head(5)
)

5 most prolific authors are :  
1. S. king
2. P.G. Wodehouse
3. Rumiko Takahashi
4. Orson Scott Card
5. Agatha Christie

Who are the 5 more prolific publisher ?

In [ ]:
(df.groupby("publisher")
 .count()
 .sort_values("title",ascending=False)
 [["title"]]
 .head(5)
)

The 5 most productive publisher are :   
1. Vintage
2. Penguin Books 
3. Penguin Classics
4. Mariner Books
5. Ballantine Books

What are the first 5 well rated titles ?

In [ ]:
(df.groupby("title")
 .agg("mean")
 .sort_values(by ="average_rating",ascending=False)
 [["average_rating"]]
 .head(5)
)

Above are the 5 well rated books title in this dataset. 

We are not going to do EDA on bookID, isbn , isbn13. 
The first 3 are identifier and have no variation, though it won't bring enough information to the model. 

Turn language code into a binary variable and perform EDA. 

In [ ]:
# bin language code  eng : 1 , other : 0
language_code_bin = []
for i in range(len(df.language_code)) :
    if re.match("^([Ee][Nn].+)",df.language_code[i]):
        language_code_bin.append(1)
    else:
        language_code_bin.append(0) 
df["language_code_bin"]  = language_code_bin

What is the proportion of books written in English language?

In [ ]:
# proportion of english in language code. 
(df.groupby("language_code_bin")
 .count()/len(df.language_code_bin)
 ).round(3)[["title"]]

English language has a proportion of 0.95 in language code. 95% of our books are written in English. 

== Finding association between variables ==  
 3.2 Numerical summary and  2D plot 


Is there any association between number of pages and average rating ?

In [ ]:
# scatter plot average rating vs num_pages
df.plot.scatter(x="num_pages",y = "average_rating")

In [ ]:
# correlation cofficient between average rating and num_pages
round(df.average_rating.corr(df.num_pages),2)

The correlation coeffication is 0.15. This suggest a weak association between 
average rating and number of pages. 

In [ ]:
# scatter plot average_rating vs text_reviews_count 
df.plot.scatter(x="text_reviews_count",y = "average_rating")

There is not a clear pattern between these 2 variables.  

In [ ]:
# correlation coef
round(df.average_rating.corr(df.text_reviews_count),2)

The correlation coef is almost 0. This suggest no association between average rating and text reviews count. 

In [ ]:
# scatter plot average rating. vs ratings count 
df.plot.scatter(x="ratings_count",y = "average_rating")

In [ ]:
# correlation coeff between average rating and ratings count 
round(df.average_rating.corr(df.ratings_count),2)

The correlation coefficient between average rating and ratings count is 0.04. This suggest that there is almost no association between average rating and rating count. 

what is the average rating of books written in english ?

In [ ]:
(df.groupby("language_code_bin")
 .agg("mean")
[["average_rating"]]
.round(2)
)

On average,books written in English are less well rated than books written in other languages.  but the the difference near 0. 

=== 2D EDA SUMMARY ===    
-numerical features are not strongly associated with average rating if considering a linear assosiation. In a linear modeling numerical features have no effect or a little one on average rating. Let's investigate other models i.e. tree based model or svm. 

Qualitative variable,i.e. language_code has on average a rating of 3.9

4 === features engineering part1  ===  
we are going to remove bookID, title , authors,isbn,isbn13, publisher because they have or present no variation and are less informative for our application.  

In [ ]:
# create a new dataframe : df1 and remove some features 
df_qualitative = df[['bookID','title','authors','isbn','isbn13',
               'publisher']]
df_qualitative

In [ ]:
df_qualitative.head(2)

4.1 Discretize average_rating

In [ ]:
# discretize average rating 
av_cat = []
av_cat = pd.cut(df.average_rating,bins=[0,1,2,3,4,5,6],
                    labels = ["0","1","2","3","4","5"],
                    include_lowest=True)
df_qualitative["av_cat"] = av_cat

In [ ]:
# head average rating 
df_qualitative.av_cat.head(3)

In [ ]:
# tail av rating 
df_qualitative.av_cat.tail(3)

In [ ]:
# proportion of classes in the data 
round(df_qualitative.av_cat.value_counts()/ len(df_qualitative.av_cat),3)

There is 40% of data in class 4 and 56% in class 3. 

In [ ]:
df_qualitative.av_cat.hist()

Discretizing average_rating  make it unbalanced data.  
We need to fix unbalalced data.

================================ REGRESSION ========================================

4.2 Splitting data 

In [ ]:
# selecting numerical features
df_final = df[['average_rating', 'num_pages','text_reviews_count', 
          "language_code_bin"]]

In [ ]:
from sklearn.model_selection import train_test_split
train0,test = train_test_split(df_final, test_size= 20,random_state=42)

In [ ]:
# split train0 in train and valid 
train,valid = train_test_split(train0,test_size=40,random_state=42)

In [ ]:
# subsetting 
# train0
X_train0 = train0.drop("average_rating",axis = 1)
y_train0 = train0.average_rating

# train
X_train = train.drop("average_rating",axis = 1)
y_train = train.average_rating

# valid 
X_valid = valid.drop("average_rating",axis=1)
y_valid = valid.average_rating

# test 
X_test = test.drop("average_rating",axis = 1)
y_test = test.average_rating

5 modelling : model testing 

5.1 Linear models

-linear model : OLS

In [ ]:
# linear model 
from sklearn import linear_model
reg = linear_model.LinearRegression()
reg.fit(X_train,y_train)


-regularized linear models

-Ridge : alpha = 0.01 

In [ ]:
# regularized model : ridge model, alpha = 0.1 
reg_ridge = linear_model.Ridge(alpha=0.01)
reg_ridge.fit(X_train,y_train)

-ridge alphas, cv = 5

In [ ]:
# ridge model with cross validation 
# regularized model : ridge model with cv =5
reg_ridge_CV = linear_model.RidgeCV(alphas=[0.001,0.01],cv=5)
reg_ridge_CV.fit(X_train0,y_train0)

-Lasso

In [ ]:
#from sklearn import linear_model, alpha = 0.5
reg_lasso = linear_model.Lasso(alpha=0.5)
reg_lasso.fit(X_train,y_train)

5.2 Tree based model 

In [ ]:
# tree based model 
from sklearn import tree
regressor = tree.DecisionTreeRegressor(random_state=40)
tree_reg= regressor.fit(X_train,y_train)

5.3 Ensemble models

In [ ]:
 #random forest
from sklearn.ensemble import RandomForestRegressor 
rf = RandomForestRegressor(n_estimators= 100,
#max_depth = 5,
random_state=42)
rf.fit(X_train,y_train)

5.4 Support vectors machine 

In [ ]:
# linear svm
from sklearn.svm import LinearSVR
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.datasets import make_regression
X, y = make_regression(n_features=4, random_state=0)
svm_regr = make_pipeline(StandardScaler(),
                     LinearSVR(random_state=0, tol=1e-5,max_iter=100000))
svm_regr.fit(X, y)

#

5.5 gbm

In [ ]:
from sklearn.ensemble import GradientBoostingRegressor
gbm = GradientBoostingRegressor(random_state=0)
gbm.fit(X_train, y_train)

In [ ]:
# cross val custom function 
def model_cross_val(models={}, X_validation=None,y_validation= None,cv=None, scoring = "neg_mean_squared_error"):
    from sklearn.model_selection import cross_val_score
    table = []
    for model in models :
        score = abs(
            cross_val_score(models[model],
                            X_validation,
                            y_validation,
                            cv=cv,
                            scoring=scoring ).mean()
        )
        score = pow(score,.5)
        table.append(score)
    print(
         pd.DataFrame(data = table,index = models,columns = ["rmse"])
        .sort_values(by ="rmse")
        
    )



 Models dictionary for performance 

In [ ]:
# models to cross-validate 

# models to cross validate
models = {"linear reg": reg,
          "ridge model":reg_ridge,
          "lasso model ": reg_lasso,
          "ridge CV":reg_ridge_CV,
          "random forest":rf,
          "linear svm":svm_regr,
           "gbm": gbm
         }

In [ ]:
# cross validation  
model_cross_val(models,X_valid,y_valid,cv= 10)

In [ ]:
# detect and remove outliers function
def remove_outliers(X,bins = 50, plot = "bool"):
    # q1 and q3 quantiles
    q1,q3 = X.quantile([0.25,0.75])

    #interquatile range 
    iqr = q3 - q1
   

    # remove  q1 - 1.5*iqr or  q3 + 1.5*iqr:
    X =  X[X.between(q1 - 1.5*iqr, q3 + 1.5*iqr,inclusive = "both")]
       

    # histogram
    if plot :
          X.hist( bins = bins,legend = {""})
    return X



In [ ]:
# remove ouliers OR NOT
q1,q3 = df.average_rating.quantile([.25,.75])
iqr = q3 - q1
#df_mdl = df_final[df_final.average_rating.between(q1 - 1.5*iqr, q3 + 1.5*iqr,inclusive = "both")]
df_mdl = df[["average_rating","num_pages","language_code_bin"]]

In [ ]:
df_mdl.head(2)

In [ ]:
df_mdl.average_rating.hist(bins = 100)


In [ ]:

remove_outliers(df_mdl.average_rating, bins = 30)


In [ ]:
remove_outliers(df_mdl.average_rating, plot = False).describe()

In [ ]:
# remove outliers from num_pages
remove_outliers(df_mdl.num_pages,bins = 30)

Removing outliers, concentrate average rating between 3.9 and 4.6. 

We end up with 11098 cases.  

In [ ]:
from sklearn.model_selection import train_test_split
# split data
train,test = train_test_split(df_mdl, test_size= 0.4,random_state=42)

# train
X_train = train.drop("average_rating",axis=1)
y_train = train.average_rating

# test
X_test = test.drop("average_rating",axis=1)
y_test = test.average_rating

In [ ]:
X_train.head(2)

In [ ]:
# linear model 
reg.fit(X_train,y_train)

In [ ]:
# stats model package
X = sm.add_constant(X_train)
mdl = sm.OLS(y_train,X_train)
fit = mdl.fit()
# print model 
print(fit.summary())

In a linear regression setting, ratings_count has a negative influence on average rating  


!!! Without outliers, we do not have multicollinearity in data in the above summary table. 

In [ ]:
# lasso
reg_lasso.fit(X_train,y_train)

In [ ]:
#ridge
reg_ridge.fit(X_train,y_train)

In [ ]:
# random forest
rf.fit(X_train,y_train)

In [ ]:
# svm
svm_regr.fit(X_train, y_train)

In [ ]:
gbm = GradientBoostingRegressor(random_state=42)
gbm.fit(X_train, y_train)
GradientBoostingRegressor(random_state=0)

7 Models new evaluation

In [ ]:
model_cross_val(models,X_validation=X_test, y_validation=y_test, cv = 10)

== Regression application : conclusion  === 
* The models performance is better without outliers.  
* Linear models are better than non-linear ones in general, but the difference is not so big than that. 
* The candidate models could be  gbm, Lasso and ridge with CV, i.e. Regularized linear regression models.

==== model evaluation : hyper parmeters search ===

In [ ]:
 # IMPORT validation curve 
from sklearn.model_selection import validation_curve

In [ ]:
# gbm hyper params search : gbm hyper-params 
gbm.get_params()

In [ ]:
# search strategy : halving random search
#from sklearn.experimental import enable_halving_search_cv  # noqa
from sklearn.model_selection import RandomizedSearchCV
#from scipy.stats import uniform

# hyper-params search
distributions = dict(alpha=[0.1,0.2,0.3,0.4,0.5],
                      learning_rate=[0.001,0.1,1,10],
                      max_depth = [1,2,3,4])

gbm_hp = RandomizedSearchCV(gbm, distributions,
                            scoring = ["max_error","r2"],
                            random_state=0,
                           refit = "max_error")
search_gbm = gbm_hp.fit(X_train, y_train)

In [ ]:
search_gbm.best_estimator_

In [ ]:
#gbm evaluation
gbm_hp2 = search_gbm.best_estimator_
                   

In [ ]:
# lasso hyper-params  list
reg_lasso.get_params()

In [ ]:
# lasso evaluation
distributions = dict(alpha=[0.1,0.2,0.3,0.4,0.5])

# hyper-params search 
lasso_hp = RandomizedSearchCV(reg_lasso, 
                              distributions, 
                              scoring = ["max_error","r2"],
                              refit = "max_error",
                              random_state=0)
search_lasso = lasso_hp.fit(X_train, y_train)
lasso_hp2 = search_lasso.best_estimator_

In [ ]:
# best lasso estimator 
lasso_hp2 

This lasso model/estimator  alpha is not different from the initial one. we hypothesize no amelioration in performance with respect to gbm estimator. 

In [ ]:
#  model evaluation
# models dictionary
models_hp = {"gbm" : gbm_hp2, 
             "lasso" :lasso_hp2}

# models evaluation 
model_cross_val(models_hp,train0,y_train0,cv = 10)

In [ ]:
# model validation : use test set
model_cross_val({"gbm":gbm_hp2},test,y_test,cv = 2)

GBM presents a rmse of 0.028 with test set.

Conclusion :  
Following, hyper-parameters search, gbm shows a far better rmse of 0.021 and  lasso a rmse of 0.28.
GBM performance improved with hyper-parameters search, while lasso did not. Therfore for tis application and in aregression setting, we would recommend gbm as final model in regression application. 

In test step (model evaluation)  gbm yields 0.028, wich is almost similar to the validation rmse. Howevr, we would recommend further , hyper-parameters serach and investigation to fine tune this model again with a large grid or grid strategy instead. 

Suggestion :
We would suggest to further perform more hyper-params search to control any overfitting issue with gbm.

In [ ]:
#vocal message 
display(Javascript("""
  var msg = new SpeechSynthesisUtterance();
  msg.text = "End of file...";
  window.speechSynthesis.speak(msg);
"""))